In [3]:
# The code was removed by Watson Studio for sharing.

# Import Necessary Libraries

In [2]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 424.68 kB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00 648.58 kB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.0-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   1.21 MB/s
branca-0.3.0-p 100% |################################| Time: 0:00:00  28.81 MB/s
vincent-0.4.4- 100% |###################

In [63]:
toronto_df = pd.read_csv(project.get_file('toronto_neigh_with_latlng.csv'))
toronto_df.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,Queen's Park,Queen's Park,M7A,43.664366,-79.392328


In [61]:
# lets get Toronto's location
addr = 'Toronto, ON'
g = Nominatim(user_agent='appjk0132')
loc = g.geocode(addr)
tlat = loc.latitude
tlng = loc.longitude
print(tlat,tlng)

43.653963 -79.387207


# Map with neighborhoods superimposed

In [64]:
map_tor = folium.Map(location=[tlat, tlng], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

# Foursquare credentials and version

In [11]:
# The code was removed by Watson Studio for sharing.

# Lets get venues in the neighborhoods from foursquare

In [55]:
# limit of number of venues returned by Foursquare API
radius = 500
nearby_venues = []

for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):    
    #build url
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius)
    
    #get json data
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    
    nearby_venues.append([(
        borough,
        neighborhood,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in venues])

In [56]:
nearby_venues_df = pd.DataFrame([item for nearby_venues in nearby_venues for item in nearby_venues])
nearby_venues_df.columns = ['Borough', 'Neighborhood', 'NeighborhoodLatitude', 'NeighborhoodLongitude', 'Venue', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

In [58]:
nearby_venues_df.groupby('Borough').count()

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
Borough,,,,,,,
Central Toronto,125,125,125,125,125,125,125
Downtown Toronto,500,500,500,500,500,500,500
East Toronto,89,89,89,89,89,89,89
East York,76,76,76,76,76,76,76
Etobicoke,69,69,69,69,69,69,69
Mississauga,11,11,11,11,11,11,11
North York,199,199,199,199,199,199,199
Queen's Park,30,30,30,30,30,30,30
Scarborough,91,91,91,91,91,91,91


In [59]:
project.save_data(data=nearby_venues_df.to_csv(index=False), file_name='toronto_neigh_nearby_venues.csv', overwrite=True)

{'asset_id': 'd163be0d-2f90-4343-871b-9d3091445e24',
 'bucket_name': 'courseracapstoneproject-donotdelete-pr-d28xfsomlxxcfy',
 'file_name': 'toronto_neigh_nearby_venues.csv',
 'message': 'File toronto_neigh_nearby_venues.csv has been written successfully to the associated OS'}

In [28]:
nearby_venues_df = pd.read_csv(project.get_file('toronto_neigh_nearby_venues.csv'))
nearby_venues_df.head()

,Borough,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,North York,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,North York,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,North York,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,North York,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [29]:
nearby_venues_df.describe(include='all')

,Borough,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
count,1354,1354,1354.000000,1354.000000,1354,1354.000000,1354.000000,1354
unique,11,100,NaN,NaN,1030,NaN,NaN,225
top,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,NaN,NaN,Subway,NaN,NaN,Coffee Shop
freq,500,30,NaN,NaN,28,NaN,NaN,96
mean,NaN,NaN,43.681681,-79.394946,NaN,43.681759,-79.394760,NaN
std,NaN,NaN,0.046202,0.068597,NaN,0.046071,0.068612,NaN
min,NaN,NaN,43.569453,-79.628939,NaN,43.567480,-79.633535,NaN
25%,NaN,NaN,43.648960,-79.419750,NaN,43.649379,-79.419654,NaN
50%,NaN,NaN,43.662696,-79.383184,NaN,43.663560,-79.383789,NaN
75%,NaN,NaN,43.709060,-79.363452,NaN,43.709225,-79.363944,NaN


# Analyze each Neighborhood

In [39]:
toronto_neighborhood['Neighborhood'] = nearby_venues_df['Neighborhood']

temp_df = pd.get_dummies(nearby_venues_df[['VenueCategory']], prefix='', prefix_sep='')

temp_df.drop(columns=['Neighborhood'], inplace=True)

toronto_neighborhood[list(temp_df.columns)] = temp_df

,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
toronto_grouped = toronto_neighborhood.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
toronto_grouped.shape

(100, 225)

# Top 10 Common Venues of each Neighborhood

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,American Restaurant,Hotel,Asian Restaurant,Seafood Restaurant,Sushi Restaurant,Food Court,Speakeasy,Smoke Shop,Greek Restaurant
1,Agincourt,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Gym,Park,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Coffee Shop,Pharmacy,Sandwich Place,Electronics Store,Eastern European Restaurant
4,"Alderwood, Long Branch",Pizza Place,Gym,Bank,Dance Studio,Pub,Sandwich Place,Coffee Shop,Athletics & Sports,Yoga Studio,Dog Run
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Park,Fried Chicken Joint,Bridal Shop,Shopping Mall,Diner,Sandwich Place,Deli / Bodega,Restaurant,Supermarket
6,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
7,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Italian Restaurant,Coffee Shop,Fast Food Restaurant,Indian Restaurant,Restaurant,Sandwich Place,Pub,Juice Bar,Thai Restaurant
8,Berczy Park,Seafood Restaurant,Café,Farmers Market,Beer Bar,Cocktail Bar,Bakery,French Restaurant,Liquor Store,Fish Market,Jazz Club
9,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


# Cluster Neighborhoods

In [102]:
clusters = 5

toronto_grouped_clustering = toronto_grouped.drop(columns=['Neighborhood'])

kmeans = KMeans(n_clusters=clusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_

array([0, 0, 1, 4, 4, 4, 0, 4, 0, 0, 0, 4, 1, 1, 4, 0, 1, 4, 0, 4, 0, 0, 0,
       0, 4, 0, 4, 0, 4, 4, 0, 4, 4, 0, 0, 0, 0, 4, 4, 4, 1, 3, 4, 0, 4, 1,
       0, 0, 0, 0, 4, 4, 0, 0, 4, 3, 4, 0, 1, 0, 4, 0, 1, 4, 0, 0, 4, 1, 4,
       0, 4, 4, 1, 4, 1, 2, 0, 4, 0, 4, 0, 0, 0, 0, 4, 4, 4, 0, 4, 1, 4, 4,
       4, 1, 0, 1, 4, 0, 0, 1], dtype=int32)

In [103]:
toronto_merged = neighborhoods_venues_sorted

toronto_merged['Cluster'] = kmeans.labels_

toronto_merged['Latitude'] = np.zeros(100)
toronto_merged['Longitude'] = np.zeros(100)

list(t_df['Neighborhood'])

['Adelaide, King, Richmond',
 'Agincourt',
 "Agincourt North, L'Amoreaux East, Milliken, Steeles East",
 'Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown',
 'Alderwood, Long Branch',
 'Bathurst Manor, Downsview North, Wilson Heights',
 'Bayview Village',
 'Bedford Park, Lawrence Manor East',
 'Berczy Park',
 'Birch Cliff, Cliffside West',
 'Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe',
 'Brockton, Exhibition Place, Parkdale Village',
 'Business reply mail Processing Centre969 Eastern',
 'CFB Toronto, Downsview East',
 'CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara',
 'Cabbagetown, St. James Town',
 'Caledonia-Fairbanks',
 'Canada Post Gateway Processing Centre',
 'Cedarbrae',
 'Central Bay Street',
 'Chinatown, Grange Park, Kensington Market',
 'Christie',
 'Church and Wellesley',
 'Clairlea, Golden Mile, Oakridge',
 "Clarks Corners, Sullivan, Tam

In [104]:
t_df = toronto_df.sort_values(by='Neighborhood')

i = 0

for k in range(100):
    if toronto_merged.iloc[k, 0] == t_df.iloc[i,1]:
        toronto_merged.iloc[k,12] = t_df.iloc[i,3]
        toronto_merged.iloc[k,13] = t_df.iloc[i,4]
        i+=1
    else:
        i+=1
        if toronto_merged.iloc[k, 0] == t_df.iloc[i,1]:
            toronto_merged.iloc[k,12] = t_df.iloc[i,3]
            toronto_merged.iloc[k,13] = t_df.iloc[i,4]
            i+=1

toronto_merged

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster,Latitude,Longitude
0,"Adelaide, King, Richmond",Steakhouse,American Restaurant,Hotel,Asian Restaurant,Seafood Restaurant,Sushi Restaurant,Food Court,Speakeasy,Smoke Shop,Greek Restaurant,0,43.650571,-79.384568
1,Agincourt,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,0,43.794200,-79.262029
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Gym,Park,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,1,43.815252,-79.284577
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Coffee Shop,Pharmacy,Sandwich Place,Electronics Store,Eastern European Restaurant,4,43.739416,-79.588437
4,"Alderwood, Long Branch",Pizza Place,Gym,Bank,Dance Studio,Pub,Sandwich Place,Coffee Shop,Athletics & Sports,Yoga Studio,Dog Run,4,43.602414,-79.543484
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Park,Fried Chicken Joint,Bridal Shop,Shopping Mall,Diner,Sandwich Place,Deli / Bodega,Restaurant,Supermarket,4,43.754328,-79.442259
6,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,0,43.786947,-79.385975
7,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Italian Restaurant,Coffee Shop,Fast Food Restaurant,Indian Restaurant,Restaurant,Sandwich Place,Pub,Juice Bar,Thai Restaurant,4,43.733283,-79.419750
8,Berczy Park,Seafood Restaurant,Café,Farmers Market,Beer Bar,Cocktail Bar,Bakery,French Restaurant,Liquor Store,Fish Market,Jazz Club,0,43.644771,-79.373306
9,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,0,43.692657,-79.264848


In [105]:
map_clusters = folium.Map(location=[tlat, tlng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i+x+(i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [108]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster'] == 0, toronto_merged.columns[:11]]
cluster2 = toronto_merged.loc[toronto_merged['Cluster'] == 1, toronto_merged.columns[:11]]
cluster3 = toronto_merged.loc[toronto_merged['Cluster'] == 2, toronto_merged.columns[:11]]
cluster4 = toronto_merged.loc[toronto_merged['Cluster'] == 3, toronto_merged.columns[:11]]
cluster5 = toronto_merged.loc[toronto_merged['Cluster'] == 4, toronto_merged.columns[:11]]

In [110]:
print('no of neighborhoods in cluster1:', cluster1.shape[0])
print('no of neighborhoods in cluster2:', cluster2.shape[0])
print('no of neighborhoods in cluster3:', cluster3.shape[0])
print('no of neighborhoods in cluster4:', cluster4.shape[0])
print('no of neighborhoods in cluster5:', cluster5.shape[0])

no of neighborhoods in cluster1: 42
no of neighborhoods in cluster2: 15
no of neighborhoods in cluster3: 1
no of neighborhoods in cluster4: 2
no of neighborhoods in cluster5: 40
